In [40]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random

In [159]:
class MapManager:
    def __init__(self):
        self.Map = self.Map_init()
        self.directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
        self.Units = []
        
        self.Teams = [self.Team(0), self.Team(1)]
        self.curr_team = 0
        self.turn_count = 0
   
        self.visited_tiles = []
    
    class Tile:
        def __init__(self, pos, unit):
            self.pos = pos
            self.unit_ref = unit
            self.tile_cost = 1.0 #Cost to move onto this tile
            
            self.move_cost = float('inf')
            self.move_parent = None
            self.visited = False
            self.is_attack = False

        def __str__(self):
            if (self.visited):
                string = '\033[94m' #Blue
                endstring = '\033[0m'
            else:
                string = ''
                endstring = ''
            if (self.unit_ref == None):
                return string + "[ ]" + endstring
            else:
                return string + "[{}]".format(self.unit_ref.Team) + endstring
            
        def __lt__(self, other):
            return self.move_cost < other.move_cost
        
        def print_tile(self):
            return "Tile ({0}, {1}):\nUnit: {2}".format(self.pos[0], self.pos[1], self.unit_ref)
        
    def __str__(self):
        string = ""
        for i in range(self.Map.shape[0]):
            for j in range(self.Map.shape[1]):
                string += str(self.Map[i,j])
            string += "\n"
        return string
                
    def Map_init(self):
        Map = np.empty([5, 5], dtype=self.Tile)
        for i in range(Map.shape[0]):
            for j in range(Map.shape[1]):
                Map[i,j] = self.Tile((i, j), None)
        return Map
    
    class Unit:
        def __init__(self, pos, cmove, hp, mmove, Att, Def, Team):
            self.pos = pos
            self.curr_move = cmove
            self.hp = hp
            
            self.max_move = mmove
            self.Att = Att
            self.Def = Def
            self.Team = Team
            
        def __str__(self):
            string = "Unit:\n\tpos: {0}\n\tcurr_move: {1}\n\thp: {2}\n".format(self.pos, self.curr_move, self.hp) 
            string += "\tmax_move: {0}\n\tAtt: {1}\n\tDef: {2}\n\tTeam: {3}\n".format(self.max_move, self.Att, self.Def, self.Team)
            return string
            
    def find_movement(self, unit):
        assert(unit is not None)
        self.dijstrkas(unit)
    
    def dijstrkas(self, unit):
        prio_queue = queue.PriorityQueue()
        Map = self.Map
        
        Map[unit.pos].move_cost = 0
        prio_queue.put(Map[unit.pos])
        self.visited_tiles.append(Map[unit.pos])
        
        while(not prio_queue.empty()):
            curr_tile = prio_queue.get()
            curr_pos = curr_tile.pos
            curr_tile.visited = True
            for i in range(4):
                new_pos = tuple(np.add(curr_pos, self.directions[i]))
                if (new_pos[0] < 0 or new_pos[0] >= self.Map.shape[0] or
                    new_pos[1] < 0 or new_pos[1] >= self.Map.shape[1] or
                    (Map[new_pos].unit_ref is not None and Map[new_pos].unit_ref.Team == unit.Team) or
                    new_pos == unit.pos or Map[new_pos].visited):
                    pass
                else:
                    new_cost = Map[new_pos].tile_cost + curr_tile.move_cost
                    if (Map[new_pos].move_cost > new_cost and new_cost <= unit.curr_move):
                        Map[new_pos].move_cost = new_cost
                        Map[new_pos].move_parent = curr_tile
                        if (Map[new_pos].unit_ref is not None):
                            Map[new_pos].is_attack = True
                        self.visited_tiles.append(Map[new_pos])
                        prio_queue.put(Map[new_pos])
                    
        
    def place_unit(self, pos, team):
        new_unit = self.Unit(pos, 10, 100, 10, 20, 10, team)
        self.Map[pos].unit_ref = new_unit
        self.Units.append(new_unit)
        self.Teams[team].units.append(new_unit)
        return new_unit
    
    def print_units(self):
        for unit in self.Units:
            print(unit)
                
    def clear_movement(self):
        for i in range(self.Map.shape[0]):
            for j in range(self.Map.shape[1]):
                self.Map[i,j].move_cost = float('inf')
                self.Map[i,j].visited = False
                self.Map[i,j].is_attack = False
        self.visited_tiles = []
        
    
    def move_unit(self, unit, pos):
        Tile = self.Map[pos]
        try:
            assert(Tile.visited)
        except AssertionError:
            print("Tile {0} not reachable by unit".format(pos))
        if(Tile.is_attack == True):
            move_tile = Tile.move_parent 
            self.Map[unit.pos].unit_ref = None
            unit.pos = move_tile.pos
            unit.curr_move -= move_tile.move_cost
            move_tile.unit_ref = unit
            
            self.combat(unit, Tile.unit_ref)
        else:
            self.Map[unit.pos].unit_ref = None
            unit.pos = pos
            unit.curr_move -= self.Map[pos].move_cost
            self.Map[pos].unit_ref = unit
        self.clear_movement()
        
    def combat(self, att_unit, def_unit):
        #att_dmg = (def_unit.Def / att_unit.Att) * 20
        def_dmg = (att_unit.Att / def_unit.Def) * 20
        #att_unit.hp -= att_dmg
        def_unit.hp -= def_dmg
        
        if (def_unit.hp <= 0):
            self.Units.remove(def_unit)
            self.Teams[def_unit.Team].units.remove(def_unit)
            self.Map[def_unit.pos].unit_ref = None
            del def_unit
    
    def Turn(self):
        if (self.curr_team == 0):
            self.curr_team = 1
        else:
            self.curr_team = 0
            self.turn_count += 1
            
        for unit in self.Units:
            if (unit.Team == self.curr_team):
                unit.curr_move = unit.max_move
    
    #returns -1 if the game is not over, else returns the number of the winning team
    def Game_result(self):
        if len(self.Teams[0].units) == 0:
            return 1
        if len(self.Teams[1].units) == 0:
            return 0
        return -1
    
    
    class Team:
        def __init__(self, num):
            self.team_num = num
            self.units = []
    

In [214]:
Map = MapManager()
unit1 = Map.place_unit((2,  2), 0)
unit2 = Map.place_unit((4,  4), 1)

#Map.find_movement(unit1)
#Map.move_unit(unit1, (4,1))

In [215]:
Map.Turn()
Map.Turn()

In [222]:
Map.find_movement(unit1)
#Map.move_unit(unit1, (4,1))
#Map.move_unit(unit1, Map.visited_tiles[random.randint(0, len(Map.visited_tiles) - 1)].pos)

input_list = []
for row in Map.Map:
    for tile in row:
        if (tile.unit_ref is not None and tile.unit_ref.Team == 1):
            input_list.append(1.0)
        else:
            input_list.append(0.0)
input_tup = tuple(input_list)
output = win_net.activate(input_tup)
move_index = np.argmax(output) 
print((move_index // 5, move_index % 5))
Map.move_unit(unit1, (move_index // 5, move_index % 5))

(4, 4)


In [223]:
print(Map)

[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][0]
[ ][ ][ ][ ][ ]



In [224]:
Map.print_units()

Unit:
	pos: (3, 4)
	curr_move: 7.0
	hp: 100
	max_move: 10
	Att: 20
	Def: 10
	Team: 0



In [114]:
input_list = []
for row in Map.Map:
    for tile in row:
        if (tile.unit_ref is not None and tile.unit_ref.Team == 1):
            input_list.append(1.0)
        else:
            input_list.append(0.0)
input_tup = tuple(input_list)

In [234]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        #genome.fitness = 1.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        games_run = 10
        wins = 0
        
        for i in range(games_run):
            Map = MapManager()
            unit1 = Map.place_unit((0,  0), 0)
            unit2 = Map.place_unit((random.randint(0, Map.Map.shape[0]) - 1,
                                    random.randint(0, Map.Map.shape[1]) - 1), 1)
            
            
            while (Map.Game_result() == -1 and Map.turn_count < 10): #Turn Count limit may have to be modified
                input_list = []
                for row in Map.Map:
                    for tile in row:
                        if (tile.unit_ref is not None and tile.unit_ref.Team == 1):
                            input_list.append(1.0)
                        else:
                            input_list.append(0.0)
                input_tup = tuple(input_list)

                #find out how to dictate output dimensions

                #use output dimensions
                output = net.activate(input_tup)
                move_index = np.argmax(output)    
                Map.find_movement(unit1)
                Map.move_unit(unit1, (move_index // 5, move_index % 5))

                #Back to computer's turn
                Map.Turn()
                Map.Turn()
            
            if (Map.Game_result() == 0):
                wins += 1
            
        #Figure out Fitness function
        genome.fitness = (wins / games_run)

In [235]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    #for xi, xo in zip(xor_inputs, xor_outputs):
    #    output = winner_net.activate(xi)
    #    print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    #node_names = {-1: 'A', -2: 'B', 0: 'A XOR B'}
    #visualize.draw_net(config, winner, True, node_names=node_names)
    #visualize.draw_net(config, winner, True, node_names=node_names, prune_unused=True)
    #visualize.plot_stats(stats, ylog=False, view=True)
    #visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)
    
    return winner_net

In [236]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config-feedforward')

In [ ]:
win_net = run(config_path)